# Step 4: PostProcess Results

This part describes how to process the optimized results for evaluation and visualization. 
1.	Prepare the following parameters.

    | Parameter | Description | Value |
    |--- |--- |--- |
    |method | Annealing method for QUBO problem |'dwave-sa': use the simulated annealer in ocean toolkit<br> 'dwave-qa': use the quantum annealer|
    |raw_path| Path for the original molecule file| './molecule-data/117_ideal.mol2' in this example |
    |data_path| Path for the processed molecule file| './qmu_117_ideal_data_latest.mol2' in this example |
    |bucket | S3 bucket to store your results | - |
    |prefix | Name of the folder in your S3 bucket | - |
    |task_id | Your quantum annealing task id| '2b5a3b05-1a0e-443a-852c-4ec422a10e59' in this example|

2. Use **ResultParser** object for SA to run the process.

In [ ]:
from utility.MoleculeParser import MoleculeData
from utility.QMUQUBO import QMUQUBO
from utility.AnnealerOptimizer import Annealer
from utility.ResultProcess import ResultParser
import time

timestamp = time.strftime("%Y%m%d-%H")

In [27]:
method = "dwave-sa"
sa_param = {}
sa_param["raw_path"] = raw_path
sa_param["data_path"] = data_path

sa_process_result = ResultParser(method, **sa_param)
# print(f"{method} result is {sa_process_result.get_all_result()}")

local_time, _ , _, _= sa_process_result.get_time()

print(f"time for {method}: \n \
    local time is {local_time}")

INFO:root:_load_raw_result
INFO:root:load simulated annealer raw result
INFO:root:MoleculeData.load()
INFO:root:init mol data for final position
INFO:root:init mol data for raw position
INFO:root:_parse_model_info
INFO:root:_init_parameters
INFO:root:parse simulated annealer result
INFO:root:sa only has local_time!


time for dwave-sa: 
     local time is 129.5549328327179


In [28]:
sa_atom_pos_data = sa_process_result.generate_optimize_pts()
# save unfold file for visualization and parameters for experiment: 1. volume value 2. relative improvement
sa_result_filepath, sa_result_json = sa_process_result.save_mol_file(f"{timestamp}")

print(f"result path is {sa_result_filepath}, and result optimization file path is {sa_result_json}")

INFO:root:generate_optimize_pts()
INFO:root:var_dict_raw {'14': ['1'], '15': ['8']} var_dict_list [{'14': '1', '15': '8'}]
INFO:root:chosen var {'x_14_1', 'x_15_8'}
INFO:root:tor list {'X_15_8', 'X_14_1'}
INFO:root:initial 18.549876807564765
INFO:root:optimize 18.56185411730613
INFO:root:optimize_gain 1.0006456813630418
INFO:root:start physical check
INFO:root:save_mol_file 20220323-14
INFO:root:finish save ./molecule-data/117_ideal_dwave-sa_20220323-14.mol2 and ./molecule-data/117_ideal_dwave-sa_20220323-14.json


result path is ./molecule-data/117_ideal_dwave-sa_20220323-14.mol2, and result optimization file path is ./molecule-data/117_ideal_dwave-sa_20220323-14.json


In [29]:
sa_process_result.parameters

{'volume': {'optimize': 18.56185411730613,
  'initial': 18.549876807564765,
  'gain': 1.0006456813630418,
  'unfolding_results': ['X_15_8', 'X_14_1'],
  'annealing_results': ['x_14_1', 'x_15_8'],
  'optimize_info': {'optimize_state': True, 'result_rank': 11}}}

As shown in the first block, the local time for SA is around 174 seconds. With the  **generate_optimize_pts()** method, the final 3D points after unfolding will be generated and saved as json file and mol2 files. 
The last block shows the optimizing results which are also stored in json files. It shows that the optimized result gains **1.0212x** increase in volume. 

3. Run the process for QA:

In [30]:
method = "dwave-qa"
qa_param = {}
qa_param["bucket"] = s3_bucket
qa_param["prefix"] = prefix
qa_param["task_id"] = qa_task_id
qa_param["raw_path"] = raw_path
qa_param["data_path"] = data_path

qa_process_result = ResultParser(method, **qa_param)
# print(f"{method} result is {qa_process_result.get_all_result()}")

local_time, task_time, total_time, access_time = qa_process_result.get_time()

print(f"time for {method}: \n \
    local time is {local_time},\n \
    task time is {task_time}, \n \
    qpu total time is {total_time}, \n \
    qpu access time is {access_time}")

INFO:root:_load_raw_result
INFO:root:load quantum annealer raw result
INFO:root:_read_result_obj
INFO:root:_read_result_obj: annealer-experiment/22419429-2f93-4679-9305-a2e099dced7c/qa_result.pickle
INFO:root:MoleculeData.load()
INFO:root:init mol data for final position
INFO:root:init mol data for raw position
INFO:root:_parse_model_info
INFO:root:_init_parameters
INFO:root:parse quantum annealer result
INFO:root:_read_result_obj
INFO:root:_read_result_obj: annealer-experiment/22419429-2f93-4679-9305-a2e099dced7c/results.json


time for dwave-qa: 
     local time is 11.894113779067993,
     task time is 3.327, 
     qpu total time is 0.144422, 
     qpu access time is 0.129189


There are many types of time metrics for running QA. This task has the local time of 7.7 s, which indicates the time between calling the API and getting the annealing result. The task time is the metric from the json file in bucket. 
The **qpu total time** and **qpu access time** represent the 
actual time running in the QPU. Refer to [Operation and Timing](https://docs.dwavesys.com/docs/latest/c_qpu_timing.html)
for details.

In [31]:
qa_atom_pos_data = qa_process_result.generate_optimize_pts()
# save unfold file for visualization and parameters for experiment: 1. volume value 2. relative improvement
qa_result_filepath, qa_result_json = qa_process_result.save_mol_file(f"{timestamp}")
print(f"result path is {qa_result_filepath}, and result optimization file path is {qa_result_json}")

INFO:root:generate_optimize_pts()
INFO:root:var_dict_raw {'14': ['1'], '15': ['8']} var_dict_list [{'14': '1', '15': '8'}]
INFO:root:chosen var {'x_14_1', 'x_15_8'}
INFO:root:tor list {'X_15_8', 'X_14_1'}
INFO:root:initial 18.549876807564765
INFO:root:optimize 18.56185411730613
INFO:root:optimize_gain 1.0006456813630418
INFO:root:start physical check
INFO:root:save_mol_file 20220323-14
INFO:root:finish save ./molecule-data/117_ideal_dwave-qa_20220323-14.mol2 and ./molecule-data/117_ideal_dwave-qa_20220323-14.json


result path is ./molecule-data/117_ideal_dwave-qa_20220323-14.mol2, and result optimization file path is ./molecule-data/117_ideal_dwave-qa_20220323-14.json


In [32]:
qa_process_result.parameters

{'volume': {'optimize': 18.56185411730613,
  'initial': 18.549876807564765,
  'gain': 1.0006456813630418,
  'unfolding_results': ['X_15_8', 'X_14_1'],
  'annealing_results': ['x_14_1', 'x_15_8'],
  'optimize_info': {'optimize_state': True, 'result_rank': 1}}}

In the same way, the optimized results are saved 
as local json and mol2 files respectively. The result indicates that QA gains **1.0006x** increase in volume.

4. Open folders for the optimized results.

With more experiments, results with timestamp are stored incrementally.

![optimize-results](../../../images/optimize-results.png)

Figure 13: Optimized results

5. Use the following method to see the original molecule and unfolding molecule.

In [37]:
# this shows the original molecule
qa_process_result.InteractView(raw_path, size=800)

interactive(children=(Text(value='./molecule-data/117_ideal.mol2', description='mol'), IntSlider(value=800, de…

In [38]:
# copy the variable for the path of unfolding molecule, we can see the unfolding results. this also works for sa_process_result.
qa_process_result.InteractView(qa_result_filepath, size=800)

interactive(children=(Text(value='./molecule-data/117_ideal_dwave-qa_20220323-14.mol2', description='mol'), In…

6. (Optional) Upload the result 117_ideal_dwave-qa_20220216-05.mol2 into
[Protein Data Bank online viewer tool](https://www.rcsb.org/3d-view) 
to see the result:

![visual](../../../images/visualization.png)

Figure 14: Protein Data Bank visualization